## Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belog in(>50k or <= 50k).

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

#### Read in the census_data.csv data with pandas

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv("census_data.csv")

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as lebels, need to use pandas.apply() method to apply a custom function that converts them to 0s and 1s.**

**Convert the Label column to 0s and 1s instead of strings.**

In [4]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [5]:
def label_fix(label):
    if label==' <=50K':
        return 0
    else:
        return 1

In [6]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

In [9]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


### Perform a Train Test Split on the Data

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
x_data = census.drop('income_bracket', axis=1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels,test_size=0.3, random_state=101)

### Create the Feature Columns for tf.estimator

** Take note of categorical vs continous values!**

In [12]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [13]:
import tensorflow as tf

**Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets.** 

In [15]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

**Create the continuous feature_columns for the continuous values using numerica_column.**

In [16]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols.**

In [17]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

## Create Input Function

In [18]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=100,
                                              num_epochs=None, shuffle=True)

### Create your model with tf.estimator 

In [19]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\Tianran\\AppData\\Local\\Temp\\tmpn5xopvt2', '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_session_config': None, '_tf_random_seed': 1, '_save_checkpoints_secs': 600}


### Train the model on the data, for at least 5000 steps.

In [20]:
model.train(input_fn=input_func, steps=5000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\Tianran\AppData\Local\Temp\tmpn5xopvt2\model.ckpt.
INFO:tensorflow:step = 1, loss = 69.31474
INFO:tensorflow:global_step/sec: 182.792
INFO:tensorflow:step = 101, loss = 324.87866 (0.551 sec)
INFO:tensorflow:global_step/sec: 254.849
INFO:tensorflow:step = 201, loss = 180.6969 (0.391 sec)
INFO:tensorflow:global_step/sec: 265.743
INFO:tensorflow:step = 301, loss = 288.04703 (0.376 sec)
INFO:tensorflow:global_step/sec: 271.439
INFO:tensorflow:step = 401, loss = 251.60435 (0.368 sec)
INFO:tensorflow:global_step/sec: 241.34
INFO:tensorflow:step = 501, loss = 33.441895 (0.415 sec)
INFO:tensorflow:global_step/sec: 269.325
INFO:tensorflow:step = 601, loss = 159.41821 (0.371 sec)
INFO:tensorflow:global_step/sec: 254.572
INFO:tensorflow:step = 701, loss = 278.20898 (0.390 sec)
INFO:tensorflow:global_step/sec: 269.672
INFO:tensorflow:step = 801, loss = 347.45895 (0.373 sec)
INFO:tensorflow:global_ste

### Evaluation

**Create a prediction input function. Remember to only support X_test data and keep shuffle=False.**

In [21]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test), shuffle=False)

** Use model.predict() and pass in the input function. This will produce a generator of predictions, which you can then transform into a list, with list()**

In [22]:
predictions = list(model.predict(input_fn=pred_fn))

INFO:tensorflow:Restoring parameters from C:\Users\Tianran\AppData\Local\Temp\tmpn5xopvt2\model.ckpt-5000


In [23]:
predictions[0]

{'class_ids': array([0], dtype=int64),
 'classes': array([b'0'], dtype=object),
 'logistic': array([0.17867997], dtype=float32),
 'logits': array([-1.5253165], dtype=float32),
 'probabilities': array([0.82132006, 0.17867999], dtype=float32)}

In [24]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [25]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

**Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data.**

In [26]:
from sklearn.metrics import classification_report

In [27]:
print(classification_report(y_test, final_preds))

             precision    recall  f1-score   support

          0       0.85      0.96      0.90      7436
          1       0.78      0.47      0.58      2333

avg / total       0.83      0.84      0.82      9769

